In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from utils.backtesting import run_backtest  # Assuming this is a custom module for backtesting

# Use correct case for rebalance frequency
rebalance_freq = "None"
include_rf = False
models_selected = []
models_selected.append("Minimum Variance")
#models_selected.append("Equal Weight")

# Function to load precomputed weights
def load_weights(method):
    # Assumes CSVs are named as 'minimum_variance_weights.csv', etc.
    path = f"data/{method}_weights.csv"
    df = pd.read_csv(path)
    # If 'Date' is not a column but is the index, reset it
    if 'Date' not in df.columns and df.index.name == 'Date':
        df = df.reset_index()
    # If wide format (Date as column, tickers as columns), set Date as index
    if 'Date' in df.columns and 'Ticker' not in df.columns:
        df = df.set_index('Date')
    return df

def load_returns():
    return pd.read_csv("data/selected_stock_daily_returns.csv", parse_dates=["Date"])

returns_df = load_returns()
performance_dict = {}
sp500_series = None

# Try to extract S&P 500 returns if present
if '^GSPC' in returns_df['Ticker'].unique():
    sp500_df = returns_df[returns_df['Ticker'] == '^GSPC'].copy()
    sp500_df = sp500_df.sort_values('Date')
    sp500_df.set_index('Date', inplace=True)
    sp500_cum = (1 + sp500_df['Daily Return']).cumprod()
    sp500_series = sp500_cum.rename('S&P 500').to_frame()

if not models_selected:
    print("No models selected for backtesting. Please select at least one model.")
else:
    for model in models_selected:
        try:
            weights_df = load_weights(model.replace(" ", "_").lower())
            # weights_df should be DataFrame with index as rebalance dates, columns as tickers, values as weights
            perf = run_backtest(returns_df, weights_df, rebalance_freq=rebalance_freq, include_rf=include_rf)
            performance_dict[model] = perf
            print(model, "backtest completed successfully.")
        except Exception as e:
            print(f"Backtest error for {model}: {e}")

    if performance_dict:
        plt.figure(figsize=(10, 5))
        for model, perf in performance_dict.items():
            plt.plot(perf.index, perf['Portfolio'], label=model)
        if sp500_series is not None:
            plt.plot(sp500_series.index, sp500_series['S&P 500'], label='S&P 500', linestyle='--')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.title('Backtest Results')
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print("Unable to compute backtest for selected models.")

Backtest error for Minimum Variance: No columns to parse from file
Unable to compute backtest for selected models.
